In [1]:
!pip install openai pinecone-client

import pinecone
import google.generativeai as genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [4]:
from pinecone import Pinecone, ServerlessSpec

# Configuring Pinecone with our API key and environment
pc = Pinecone(api_key='YOUR_PINECONE_API_KEY')

PINECONE_INDEX_NAME = 'chat-history'

if PINECONE_INDEX_NAME not in pc.list_indexes():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(PINECONE_INDEX_NAME)

# Configuring Gemini with it's API key and generation configuration
genai.configure(api_key='YOUR_GEMINI_API_KEY')

# Generation configuration
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

# GenerativeModel instance
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)

In [5]:
history = [
  "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
  "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
  "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
  "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
  "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
  "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
  "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
  "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
  "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
  "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
  "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
  "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
  "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
  "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
  "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
  "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
  "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
  "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
  "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
  "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]


In [27]:
def add_embeddings_to_pinecone(history, index_name='chat-history'):

    if index_name not in pc.list_indexes():
        pc.create_index(
            name=index_name,
            dimension=768,
            metric="cosine",
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

    # Embedding conversation history using Gemini
    embeddings = genai.embed_content(model="models/text-embedding-004", content=history)['embedding']

    vectors=[]
  # Add our embeddings to Pinecone
    for i, embedding in enumerate(embeddings):
        # Each vector is a tuple which contains the (id, values, metadata)
        vectors.append((f'message-{i+1}', embedding, {'text': history[i]}))

    # Upserting the vectors into Pinecone
    index.upsert(vectors)
add_embeddings_to_pinecone(history, index_name=PINECONE_INDEX_NAME)

In [28]:
def retrieve_relevant_history(query, history, index_name='chat-history'):
    index = pc.Index(index_name)

    # Encoding the query
    query_embedding = genai.embed_content(model="models/text-embedding-004", content=[query])['embedding'][0]

    # Querying the Pinecone for relevant history
    results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

    # Just a small debugging step: Print the results to see if any matches are found
    print(f"Query results: {results}")

    # Retrieving the most relevant history messages
    relevant_messages = [match['metadata']['text'] for match in results['matches'] if 'metadata' in match]

    return relevant_messages



In [29]:
def prepare_prompt(test_prompt, history, index_name='chat-history'):
    """
    This function:
    1. Retrieves relevant history messages using the RAG mechanism.
    2. Combines the retrieved messages with the test prompt.
    3. Ensures the combined prompt does not exceed the maximum token limit.
    4. Returns the combined prompt and the context referred.
    """
    # Retrieving relevant history using the RAG mechanism
    relevant_history = retrieve_relevant_history(test_prompt, history, index_name)

    # Combining the retrieved history with the test prompt
    combined_prompt = "\n".join(relevant_history) + f"\nUser: {test_prompt}\nAssistant:"

    # Let's assume our max token limit is 4096
    if len(combined_prompt.split()) > 4096:
        combined_prompt = ' '.join(combined_prompt.split()[:4096])

    return combined_prompt, relevant_history



In [31]:
def test_final_prompt():
    """
    This function:
    1. Defines the final test prompt.
    2. Prepares the prompt using the prepare_prompt function.
    3. Calls OpenAI to generate a response.
    4. Prints the final test prompt, the chat history context referred to, and the final response.
    5. Validates manually if the final response references the correct context.
    """
    final_test_prompt = "Do you think it will help me stay fit?"

    # Preparing the prompt and retrieving the context
    prepared_prompt, context_referred = prepare_prompt(final_test_prompt, history)

    # Generating a response from the Gemini model
    response = model.generate_content(contents=prepared_prompt)

    print(f"Final Test Prompt: {final_test_prompt}")
    print(f"Context Referred: {context_referred}")
    print(f"Final Test Prompt Response: {response.candidates[0].content.parts[0].text}")


test_final_prompt()


Query results: {'matches': [{'id': 'message-7',
              'metadata': {'text': '7: User: Perfect! Now, what are some good '
                                   'exercises? | Bot: You can try a mix of '
                                   'cardio and strength training exercises.'},
              'score': 0.471715927,
              'values': []},
             {'id': 'message-8',
              'metadata': {'text': '8: User: Any specific recommendations for '
                                   'cardio? | Bot: Running, cycling, and '
                                   'swimming are all excellent cardio '
                                   'exercises.'},
              'score': 0.430772066,
              'values': []},
             {'id': 'message-9',
              'metadata': {'text': "9: User: I'll start with running. Can you "
                                   "recommend any books? | Bot: 'Atomic "
                                   "Habits' by James Clear is a highly "
                